# *import*

In [1]:
# from collections import OrderedDict
from typing import List, Tuple, Callable
import matplotlib.pyplot as plt
from datasets.utils.logging import enable_progress_bar
enable_progress_bar()
import time
import threading
import random

import torch


import flwr
from flwr.client import Client, ClientApp
from flwr.common import ndarrays_to_parameters, Context, Metrics
from flwr.server import ServerApp, ServerConfig, ServerAppComponents

from flwr.simulation import run_simulation

# from utils.model_CNN import Net
from utils.model_CNN import SVHNNet as Net
# from utils.model_CNN import ImprovedSVHNNet as Net
from utils.train_test import test
from utils.loaddata import get_cached_datasets
from utils.others import get_parameters, set_parameters, evaluate_and_plot_confusion_matrix


from utils.history import history
from utils.client import FlowerClient
from utils.clientmanger import DynamicClientManager
from utils.HRFA_strategy import HRFA
from utils.other_strategy import AdaFedAdamStrategy, MyFedAvg

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.5.1+cu124


# **Load Data**

In [2]:
NUM_CLIENTS = 100
FRACTION = 1
BATCH_SIZE = 64
NUM_ROUNDS = 500
NUM_EPOCHS = 5
NUM_WORKERS = 8
VERBOSE = False
ATTACK_NUMS = 0
ATTACK_TYPE = "UPA"
DFL = False
Q = 0.9
DATASET = "svhn" #"svhn", "cifar10", "cifar100", "svhn"

In [3]:
# trainloader, _, _ = get_cached_datasets(partition_id=0, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)
# batch = next(iter(trainloader))
# images, labels = batch["img"], batch["label"]

# # Reshape and convert images to a NumPy array
# # matplotlib requires images with the shape (height, width, 3)
# images = images.permute(0, 2, 3, 1).numpy()

# # Denormalize
# images = images / 2 + 0.5

# # Create a figure and a grid of subplots
# fig, axs = plt.subplots(4, 8, figsize=(12, 6))

# # Loop over the images and plot them
# for i, ax in enumerate(axs.flat):
#     ax.imshow(images[i])
#     ax.set_title(trainloader.dataset.features["label"].int2str([labels[i]])[0])
    
#     ax.axis("off")

# # Show the plot
# fig.tight_layout()
# plt.show()

# **my fumction**

## **dynamic control**

In [4]:
current_server_round = 0
new_round_event = threading.Event()

def get_current_round() -> int:
    return current_server_round

In [5]:
def base_on_off(
    client_manager: DynamicClientManager,
    current_round,
    online_list: List[str],
    offline_list: List[str],
):
    # 下線部分在線客戶端        
    if current_round<4 and online_list and DFL:
        offline_num = 1
        for cid in random.sample(online_list, offline_num):
            client = client_manager.clients.get(cid)
            if client:
                client_manager.unregister(client)

    # 上線部分離線客戶端
    if current_round>3 and offline_list and DFL:
        online_num = 1
        for cid in random.sample(offline_list, online_num):
            client = client_manager.off_clients.get(cid)
            if client:
                print(f"[TOGGLER] 將 {cid} 上線")
                client_manager.register(client)

In [6]:
def random_on_off(
    client_manager: DynamicClientManager,
    current_round,
    online_list: List[str],
    offline_list: List[str],
    offline_rate: float = 0.3,
    online_rate: float = 0.3
):
    try:
        # 確保列表不為空
        if not online_list or not offline_list:
            print(f"警告: 客戶端列表為空 (在線:{len(online_list)}, 離線:{len(offline_list)})")
        
        # 下線部分在線客戶端
        if current_round % 2 != 0 and online_list and DFL:
            # 確保取樣數量不超過列表長度
            # offline_num = min(1, len(online_list))
            offline_num = 1
            if offline_num > 0:
                for cid in random.sample(online_list, offline_num):
                    client = client_manager.clients.get(cid)
                    if client:
                        client_manager.unregister(client)
        
        # 上線部分離線客戶端
        if current_round % 2 == 0 and offline_list and DFL:
            # 確保取樣數量不超過列表長度
            # online_num = min(1, len(offline_list))
            online_num = 1
            if online_num > 0:
                for cid in random.sample(offline_list, online_num):
                    client = client_manager.off_clients.get(cid)
                    if client:
                        print(f"[TOGGLER] 將 {cid} 上線")
                        client_manager.register(client)
    except Exception as e:
        print(f"random_on_off 執行錯誤: {e}")
        import traceback
        traceback.print_exc()

In [7]:
def background_online_offline_simulator(
    client_manager: DynamicClientManager,
    get_round: Callable[[], int],
    event: threading.Event,
    # interval: int = 30,
    offline_rate: float = 0.3,
    online_rate: float = 0.2
):
    while True:
        event.wait()  # 等待 server 通知新的一輪開始
        # time.sleep(interval)
        current_round = get_round()
        
        online_list = list(client_manager.online_clients)
        offline_list = list(client_manager.offline_clients)

        # base_on_off(client_manager, current_round, online_list, offline_list)
        random_on_off(client_manager, current_round, online_list, offline_list, offline_rate, online_rate)

        event.clear()  # 清除標記，等待下一次 signal

# **train&test**

# **Client**

In [8]:
def client_fn(context: Context) -> Client:
    # Load model
    net = Net().to(DEVICE, memory_format=torch.channels_last)
    
	# Read the node_config to fetch data partition associated to this node
    partition_id = str(context.node_config["partition-id"])  # 強制轉換為字串
    num_partitions = context.node_config["num-partitions"]
    trainloader, valloader, _ = get_cached_datasets(partition_id, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)

    if int(partition_id) < ATTACK_NUMS:
        return FlowerClient(partition_id, net, trainloader, valloader, ATTACK_TYPE).to_client()
    else:
        return FlowerClient(partition_id, net, trainloader, valloader, None).to_client()

In [9]:
# Create the ClientApp
client = ClientApp(client_fn=client_fn)

# **Server**

### setting

In [10]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = []
    examples = []
    for num_examples, m in metrics:
        if "client_accuracy" not in m:
            print(f"Warning: Missing 'client_accuracy' in metrics for client")
            continue
        accuracies.append(num_examples * m["client_accuracy"])
        examples.append(num_examples)
    total_examples = sum(examples)
    if total_examples <= 0:
        print("Warning: No valid examples for aggregation. Returning 0.0")
        return {"accuracy": 0.0}
    return {"accuracy": sum(accuracies) / total_examples}

In [11]:
def server_evaluate(server_round, parameters, config):
    global current_server_round
    # 更新全域變數，讓背景執行緒知道目前是第幾個 round
    current_server_round = server_round
    """Evaluate the global model after each round (不再畫 confusion matrix)."""
    start_time = time.time()  # 記錄開始時間
    net = Net().to(DEVICE)
    set_parameters(net, parameters)

    # 加載測試集
    _, _, testloader = get_cached_datasets(0, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)

    # 測試
    loss, accuracy = test(net, testloader)
    
    end_time = time.time()  # 記錄結束時間
    round_time = end_time - start_time  # 計算 round 時間
    
    history.add_loss_centralized(server_round, loss)
    history.add_metrics_centralized(server_round, {"accuracy": accuracy})

    # 只記錄最終模型，不畫 confusion matrix
    if server_round == NUM_ROUNDS:  # 最後一輪才返回模型
        evaluate_and_plot_confusion_matrix(net, testloader, DEVICE)
        return loss, {"accuracy": accuracy}
    new_round_event.set()
    return loss, {"accuracy": accuracy}


In [12]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        # "local_epochs": 1 if server_round < 2 else NUM_EPOCHS,
        "local_epochs": NUM_EPOCHS,
        "train_mode": "lookahead",
    }
    return config

In [13]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())
param_count = sum(p.numel() for p in Net().parameters() if p.requires_grad)
print(f"Trainable Parameters: {param_count:,d}")
# _, _, testloader = get_cached_datasets(0, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)
# Create FedAvg strategy
strategy = HRFA(
    fraction_fit=FRACTION,  # Sample 100% of available clients for training
    fraction_evaluate=FRACTION,  # Sample 50% of available clients for evaluation
    
    min_fit_clients=2,  # Never sample less than 10 clients for training
    min_evaluate_clients=2,  # Never sample less than 5 clients for evaluation
    min_available_clients=2,  # Wait until all 10 clients are available
    
    initial_parameters=ndarrays_to_parameters(params),  # Pass initial model parameters
    evaluate_fn=server_evaluate,  # 設定 evaluate_fn
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    on_fit_config_fn=fit_config,  # Pass the fit_config function
    on_evaluate_config_fn=fit_config,
    net=Net().to(DEVICE),
    # testloader = testloader,
)

Trainable Parameters: 627,210


In [14]:
# # Create an instance of the model and get the parameters
# params = get_parameters(Net())
# param_count = sum(p.numel() for p in Net().parameters() if p.requires_grad)
# print(f"Trainable Parameters: {param_count:,d}")

# # Create FedAvg strategy
# strategy = AdaFedAdamStrategy(
#     net=Net(),
#     fraction_fit=FRACTION,
#     fraction_evaluate=FRACTION,
#     min_fit_clients=NUM_CLIENTS,
#     min_evaluate_clients=NUM_CLIENTS,
#     min_available_clients=NUM_CLIENTS,
#     initial_parameters=ndarrays_to_parameters(params),
#     evaluate_fn=server_evaluate,
#     evaluate_metrics_aggregation_fn=weighted_average,
#     on_fit_config_fn=fit_config,
#     on_evaluate_config_fn=fit_config,
# )

In [15]:
# # Create an instance of the model and get the parameters
# params = get_parameters(Net())
# param_count = sum(p.numel() for p in Net().parameters() if p.requires_grad)
# print(f"Trainable Parameters: {param_count:,d}")
# # Create FedAvg strategy
# strategy = MyFedAvg(
#     fraction_fit=FRACTION,  # Sample 100% of available clients for training
#     fraction_evaluate=FRACTION,  # Sample 50% of available clients for evaluation
#     min_fit_clients=NUM_CLIENTS,  # Never sample less than 10 clients for training
#     min_evaluate_clients=NUM_CLIENTS,  # Never sample less than 5 clients for evaluation
#     min_available_clients=NUM_CLIENTS,  # Wait until all 10 clients are available
#     initial_parameters=ndarrays_to_parameters(params),  # Pass initial model parameters
#     evaluate_fn=server_evaluate,  # 設定 evaluate_fn
#     evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
#     on_fit_config_fn=fit_config,  # Pass the fit_config function
#     on_evaluate_config_fn=fit_config
# )

### Define Server

In [16]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour."""
    global testloader
    _, _, testloader = get_cached_datasets(0, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)

    # 設定 ServerConfig，如同你的程式碼
    config = ServerConfig(num_rounds=NUM_ROUNDS)

    # 建立動態管理器
    client_manager = DynamicClientManager()
        
    # 建立並啟動背景執行緒，模擬客戶端動態上/下線
    simulator_thread = threading.Thread(
        target=background_online_offline_simulator,
        args=(client_manager, get_current_round, new_round_event),  # interval=30秒, toggle_rate=0.3
        daemon=True  # 設 daemon=True 可以在主程式結束時自動退出
    )
    simulator_thread.start()

    return ServerAppComponents(
        strategy=strategy,
        config=config,
        client_manager=client_manager,
    )


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# *Run simulation*

### Run setting

In [17]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 2 , 
                                           "num_gpus": 0.2 #if (NUM_CLIENTS*FRACTION)>40 else 2/(NUM_CLIENTS*FRACTION)
                                          }}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [18]:
import os

# 1. 設定 Ray 要用的暫存目錄
os.environ["TMPDIR"] = "/home/koshino17/koshino17_hdd/tmp"

In [ ]:
# 讓 Flower 運行完整的 FL 訓練
start_time = time.time()  # 記錄開始時間

run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

end_time = time.time()  # 記錄結束時間
total_time = end_time - start_time  # 計算總時間

print(f"Total Training Time: {total_time:.2f} seconds")  # 顯示總時間

[Sat Jun 21 14:20:03 2025] 初始化聯邦數據集: svhn, 分區數=100, q=0.9


INFO :      Starting Flower ServerApp, config: num_rounds=500, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


[Status] Client 13391035250180190972 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 1240965848916339717 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 3694980021011020811 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 14720443434614230285 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 18325805644163320846 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 6927540652876234516 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 2363219094045783836 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 7750653997537419547 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 8902451577454289182 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 4589640094243324453 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 8836029881398553638 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 6633322374520119339 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 2339253131485391406 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 17362468309046208041 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓

INFO :      initial parameters (loss, other metrics): 0.03589136623757415, {'accuracy': 0.12373232944068838}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)


警告: 客戶端列表為空 (在線:100, 離線:0)
(ClientAppActor pid=3273264) [Sat Jun 21 14:21:00 2025] 初始化聯邦數據集: svhn, 分區數=100, q=0.9


...

INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [ROUND 31]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)
INFO :      aggregate_fit: received 100 results and 0 failures
INFO :      fit progress: (31, 0.027563851750292658, {'accuracy': 0.39197910264290103}, 1647.407936588017)
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)


警告: 客戶端列表為空 (在線:100, 離線:0)


INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [ROUND 32]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)
INFO :      aggregate_fit: received 100 results and 0 failures
INFO :      fit progress: (32, 0.026521130095762055, {'accuracy': 0.41180086047940995}, 1698.6195638630015)
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)


警告: 客戶端列表為空 (在線:100, 離線:0)


INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [ROUND 33]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)


### Loss 和 Accuracy 折線圖

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 提取每個 client（按 pid）的 loss 和 accuracy 數據
client_losses = {}  # {pid: [(round, loss), ...]}
client_accuracies = {}  # {pid: [(round, acc), ...]}

for round_num, pid, loss in history.losses_distributed:
    if pid not in client_losses:
        client_losses[pid] = []
    client_losses[pid].append((round_num, loss))

for round_num, pid, acc in history.metrics_distributed.get('accuracy', []):
    if pid not in client_accuracies:
        client_accuracies[pid] = []
    client_accuracies[pid].append((round_num, acc))

# 為圖表生成隨機顏色
colors = plt.cm.tab20(np.linspace(0, 1, max(len(client_losses), len(client_accuracies))))

# 繪製每個 client 的 Loss 圖
plt.figure(figsize=(10, 6))
for idx, (pid, losses) in enumerate(client_losses.items()):
    rounds = [r for r, _ in sorted(losses)]
    loss_values = [l for _, l in sorted(losses)]
    plt.plot(rounds, loss_values, marker='o', color=colors[idx], label=f'Partition {pid}', alpha=0.7)

# 繪製 centralized 和 distributed loss
rounds_loss = [t[0] for t in history.losses_centralized]
loss_values_centralized = [t[1] for t in history.losses_centralized]

distributed_loss_by_round = {}
for round_num, _, loss in history.losses_distributed:
    if round_num not in distributed_loss_by_round:
        distributed_loss_by_round[round_num] = []
    distributed_loss_by_round[round_num].append(loss)
rounds_loss_distributed = sorted(distributed_loss_by_round.keys())
loss_values_distributed = [np.mean(distributed_loss_by_round[r]) for r in rounds_loss_distributed]

plt.plot(rounds_loss, loss_values_centralized, marker='o', color='red', label='Centralized Loss', linewidth=2, linestyle='--')
# plt.plot(rounds_loss_distributed, loss_values_distributed, marker='o', color='orange', label='Distributed Loss', linewidth=2, linestyle='--')

plt.xlabel('Round')
plt.ylabel('Loss')
plt.title('Loss Over Rounds (Per Client by PID)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# 繪製每個 client 的 Accuracy 圖
plt.figure(figsize=(10, 6))
for idx, (pid, accuracies) in enumerate(client_accuracies.items()):
    rounds = [r for r, _ in sorted(accuracies)]
    acc_values = [a for _, a in sorted(accuracies)]
    plt.plot(rounds, acc_values, marker='o', color=colors[idx], label=f'Partition {pid}', alpha=0.7)

# 繪製 centralized 和 distributed accuracy
acc_rounds = [t[0] for t in history.metrics_centralized.get('accuracy', [])]
acc_values_centralized = [t[1] for t in history.metrics_centralized.get('accuracy', [])]

distributed_acc_by_round = {} 
for round_num, _, acc in history.metrics_distributed.get('accuracy', []):
    if round_num not in distributed_acc_by_round:
        distributed_acc_by_round[round_num] = []
    distributed_acc_by_round[round_num].append(acc)
acc_rounds_md = sorted(distributed_acc_by_round.keys())
acc_values_md = [np.mean(distributed_acc_by_round[r]) for r in acc_rounds_md]

plt.plot(acc_rounds, acc_values_centralized, marker='o', color='blue', label='Centralized Accuracy', linewidth=2, linestyle='--')
# plt.plot(acc_rounds_md, acc_values_md, marker='o', color='green', label='Distributed Accuracy', linewidth=2, linestyle='--')

plt.xlabel('Round')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Rounds (Per Client by PID)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ========= 訓練結束後繪圖 =========

# 假設 NUM_ROUNDS 為全局訓練輪數
rounds = list(range(1, NUM_ROUNDS + 1))
metrics_to_plot = ['reputation', 'short_term_rs', 'long_term_rs', 'similarity', 'shapley']

# 從策略中提取 metric_history
metric_history = strategy.metric_history

for metric in metrics_to_plot:
    plt.figure(figsize=(10, 6))
    for partition_id, metrics_dict in metric_history.items():
        # if partition_id == -1:
        #     continue
        values = metrics_dict[metric]
        # 若某 client 的紀錄少於 NUM_ROUNDS (代表部分 round 沒有更新)，則補上最後一次的值
        if len(values) < NUM_ROUNDS:
            last_val = values[-1] if values else 0.0
            values = values + [last_val] * (NUM_ROUNDS - len(values))
        plt.plot(rounds, values, marker='o', label=f'Partition {partition_id}')
    plt.xlabel('Round')
    plt.ylabel(metric)
    plt.title(f'{metric} over Rounds')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
history.metrics_centralized